## Data Ingestion & Preparation

This notebook pulls Covid-19 data on the number of cases and deaths by state and merges it to Google mobility data.

A file, **covid19_google_mobility_by_state.csv**, is saved after running the notebook and contains 

|Column | Description |
|--------|--------|
|`state` | State |
|`date` | Date | 
|`retail_and_recreation` | Google Mobility: Percent change from base, retail and recreation |
|`grocery_and_pharmacy` | Google Mobility: Percent change from base, grocery and pharmacy |
|`parks` | Google Mobility: Percent change from base, parks |
|`transit_stations` | Google Mobility: Percent change from base, transit stations |
|`workplaces` | Google Mobility: Percent change from base, workplaces |
|`residential` | Google Mobiity: Percent change from base, residential |
|`pop` | State population (from Covid-19 deaths data) |
|`deaths` | Number of deaths to date (from Covid-19 deaths data)|
|`cases` | Number of confirmed cases to date (from Covid-19 confirmed cases data)|
|`cases_per_capita` | Generated: Number of cases/population|
|`deaths_per_capita` | Generated: Number of deaths/population|
|`death_rate` | Generated: Number of deaths/number of cases|
|`dcases` | Generated: Change in number of confirmed cases|
|`ddeaths` | Generated: Change in number of deaths|


### Data are pulled from the following links

* Google Mobility: https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv
* JH Cases: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv
* JH Deaths: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv



Adding additional variables from alternate sources to the state-level dataset:


* Policy data  by state: https://github.com/KristenNocka/COVID-19-US-State-Policy-Database

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import sys
import os
import datetime
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


plt.style.use('seaborn')
plt.rcParams['figure.figsize']=(10, 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import warnings
warnings.filterwarnings('ignore')

## Now we can move on to the mobility and covid data

In [ ]:
url_cases = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
url_mobility = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

## Pull Google Mobility Data

A few notes on the Google Mobility data before we dive in. 

The file we pull can be found at https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv and contains mobility data for the following categories (variable names in parentheses):

* Retail and recreation (`retail_and_recreation`)
* Grocery and pharmacy (`grocery_and_pharmacy`)
* Parks (`parks`)
* Transit stations (`transit_stations`)
* Workplaces (`workplaces`)
* Residential (`residential`)

We are going to create three separate data sets:

1. National level
2. State level
3. County level

The raw data is at the global level - we trim it down to just observations for the United States. After doing so, the variables `sub_region_1` and `sub_region_2` are *state* and *county*, respectively. 

- Observations missing **both** `sub_region_1` **and** `sub_region_2` are for the United States as a whole
- Observations missing `sub_region_2` and **NOT** `sub_region_1` are at the state (and DC) level
- Observations with **both** `sub_region_1` and `sub_region_2` are at the county level

We will keep this in mind as we create the three data sets.

In [ ]:
# Read in the full goog-mob file.
google_mobility = pd.read_csv(url_mobility)

# Trim to the US and rename columns
# 6/16 added drop fips and iso
google_mobility = google_mobility[google_mobility['country_region_code']=='US'].drop(columns = {'country_region_code', 'country_region', 'iso_3166_2_code','census_fips_code'})
# google_mobility = google_mobility[google_mobility['country_region_code']=='US'].drop(columns = {'country_region_code', 'country_region','iso_3166_2_code'})

google_mobility = google_mobility.rename(columns = {
    'retail_and_recreation_percent_change_from_baseline': 'retail_and_recreation', 
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_and_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations',
    'workplaces_percent_change_from_baseline': 'workplaces',
    'residential_percent_change_from_baseline': 'residential',
    'sub_region_1': 'state',
    'sub_region_2': 'county'})

# Create US-level data frame
google_mobility_us = google_mobility[google_mobility['state'].isna() & google_mobility['county'].isna()].drop(columns={'state', 'county'})
google_mobility_us['date'] = pd.to_datetime(google_mobility_us['date'], format="%Y-%m-%d")
google_mobility_us = google_mobility_us.set_index('date')

# Create state-level data frame
google_mobility_states = google_mobility[google_mobility['county'].isna() & ~google_mobility['state'].isna()].drop(columns={'county'})
google_mobility_states['date'] = pd.to_datetime(google_mobility_states['date'], format="%Y-%m-%d")
state_date = google_mobility_states[['state', 'date']]
google_mobility_states = google_mobility_states.set_index(pd.MultiIndex.from_frame(state_date))
google_mobility_states.head()

## Pull Johns Hopkins Data on Confirmed Cases

For now this is just at the level of the state - I am going to do it by county next

In [ ]:
# Read CSV from GitHub repo
cases = pd.read_csv(url_cases)

# Drop stuff we don't need and trim to the 50 states and DC
cases = cases.rename(columns = {'Admin2': 'county', 'Province_State': 'state'})
cases = cases.drop(columns={'county', 'iso2', 'iso3', 'code3', 'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'UID', 'FIPS'})
drop_states = ['Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands', 'Guam', 'American Samoa', 'Diamond Princess', 'Grand Princess']
for st in drop_states:
    cases = cases[cases['state']!=st]

# Reshape to long format
cases = cases.melt(id_vars = 'state', var_name = 'date', value_name='cases').sort_values(by = ['state', 'date'])
cases = cases.groupby(['state', 'date']).sum().reset_index()

# Set the date variable to datetime and sort the data frame
cases['date'] = pd.to_datetime(cases['date'], format="%m/%d/%y")
cases = cases.sort_values(['state', 'date'])
cases.head()

## Pull Johns Hopkins Data on Deaths

Again, just at the state-level...for now.

In [ ]:
# Read CSV from GitHub repo
deaths = pd.read_csv(url_deaths)

# Drop stuff we don't need and trim to the 50 states and DC
deaths = deaths.drop(columns={'iso2', 'iso3', 'code3', 'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'UID', 'FIPS'})
drop_states = ['Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands', 'Guam', 'American Samoa', 'Diamond Princess', 'Grand Princess']
for st in drop_states:
    deaths = deaths[deaths['Province_State']!=st]

# Rename some variables and make sure the states look ok
deaths = deaths.rename(columns = {'Admin2': 'county', 'Province_State': 'state', 'Population': 'pop'})
deaths['state'].value_counts()
deaths = deaths.sort_values('state')
deaths = deaths.drop(columns='county')

# Reshape to long format
deaths = deaths.melt(id_vars = {'state', 'pop'}, var_name = 'date', value_name='deaths').sort_values(by = ['state', 'date'])
deaths = deaths.groupby(['state', 'date']).sum().reset_index()

# Set the date variable to datetime and sort the data frame
deaths['date'] = pd.to_datetime(deaths['date'], format="%m/%d/%y")
deaths = deaths.sort_values(['state', 'date'])
deaths.head()

## Merge the Confirmed Cases and Deaths Data Frames

In [ ]:
# Merge and reset the index
covid19 = deaths.merge(cases)
covid19 = covid19.set_index(pd.MultiIndex.from_frame(covid19[['state', 'date']]))

# Drop state and date variables (they're in the index)
covid19 = covid19.drop(columns={'state', 'date'})

# Daily cases_per_capita
# covid19['cases_per_capita'] = covid19['cases'].divide(covid19['pop'])
# covid19['cases_per_capita'] = covid19['cases'].diff()
covid19['cases_per_capita'] = covid19['cases'].divide(covid19['pop'])
covid19['cases_per_capita'] = covid19['cases_per_capita'].diff()

# Generate deaths per capita
covid19['deaths_per_capita'] = covid19['deaths'].divide(covid19['pop'])

# And a "death-rate" as the number of deaths/confirmed cases
covid19['death_rate'] = covid19['deaths'].divide(covid19['cases'])

# Change in number of cases
covid19['dcases'] = covid19['cases'].diff()

# Change in number of deaths
covid19['ddeaths'] = covid19['deaths'].diff()
covid19.tail()

## Merge Covid-19 Data with Google Mobility Data

Then, write the data frame to a CSV

In [ ]:
# Merge the two data frames on their indexes, which we set above
df = google_mobility_states.merge(covid19, left_on=google_mobility_states.index.to_numpy(), right_on=covid19.index.to_numpy()).reset_index()

# Now reset the index
df = df.set_index(pd.MultiIndex.from_frame(df[['state', 'date']]))

# Drop stuff we don't need 
df = df.drop(columns={'index', 'key_0', 'state', 'date', 'metro_area'})
df.to_csv('covid19_google_mobility_by_state.csv')
df.head()
df.tail()

#### End of Data Engineering

In [ ]:
df

# Exploratory Data Analysis

## Why we are focusing on state level data
- America in Covid is like 50 different petri dishes, no 2 states are alike in policy or results
- Initial assumption is State level policy is driving mobility so we begin our analysis at the state level rather than country or county
- First step is to find states that have seen a change since the economy has "reopened"
- We begin by examining some moving averages

### Bokeh trial

In [ ]:
# Plotting was easier with this change
df1 = df.reset_index(level=['state','date'])
df1.tail()

In [ ]:
df1.state = df1.state.astype(str)
df1.cases = df1.dcases.astype(str)

In [ ]:
group = df1.groupby(by=['state','dcases'])
group

### most recent date data fram

In [ ]:
recent_date = df1['date'].max()
df_latest = df1[df1['date'] == recent_date]


In [ ]:
df_latest

### Eventually we can use this framework for a webapp, need plots first

In [ ]:
# # Pandas for data management
# import pandas as pd

# # os methods for manipulating paths
# from os.path import dirname, join

# # Bokeh basics 
# from bokeh.io import curdoc
# from bokeh.models.widgets import Tabs


# # Each tab is drawn by one script
# # from scripts.histogram import histogram_tab
# # from scripts.density import density_tab
# # from scripts.table import table_tab
# # from scripts.draw_map import map_tab
# # from scripts.routes import route_tab

# # Using included state data from Bokeh for map
# from bokeh.sampledata.us_states import data as states

# # Read data into dataframes
# # flights = pd.read_csv(join(dirname(__file__), 'data', 'flights.csv'), 
# # 	                                          index_col=0).dropna()
# daily_cases = df_latest['dcases']
# overall_cases = df_latest['cases']

# # Formatted Flight Delay Data for map
# # map_data = pd.read_csv(join(dirname(__file__), 'data', 'flights_map.csv'),
# #                             header=[0,1], index_col=0)

# # Create each of the tabs
# # tab1 = histogram_tab(daily_cases)
# # tab2 = density_tab(daily_cases)
# # tab3 = table_tab(daily_cases)
# # tab4 = map_tab(daily_cases, states)
# # tab5 = route_tb(daily_cases)

# # Put all the tabs into one application
# tabs = Tabs(tabs = [tab1, tab2, tab3, tab4, tab5])

# # Put the tabs in the current document for display
# curdoc().add_root(tabs)

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

group = df_latest.groupby(by=['state','dcases'])
source = ColumnDataSource(group)

p = figure(plot_width=800, plot_height=300, title="Daily_Cases_by_State",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "States grouped by # Cases"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('states', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df_latest.state.unique()), end=1)

p.vbar(x='states', top='cases', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("Cases", "@dcases")]))

show(p)

# Mobility EDA
## Charting mobility and daily cases
### Starting with Falling Cases states

In [ ]:
# Function to speed up plotting 10 different states
def plot_mobility_cases(state_name):
    """
    For given state or states, returns moblity charts and new daily cases chart
    """
    plt.figure()
    plt.title('Moblity ' + state_name)
    df.loc[state_name]['retail_and_recreation'].plot(label='RetailRec')
    df.loc[state_name]['residential'].plot(label='Residential')
    df.loc[state_name]['workplaces'].plot(label='Workplace')
    df.loc[state_name]['transit_stations'].plot(label='Transit')
    plt.legend()

    plt.figure()
    plt.title('Daily cases ' + state_name)
    df.loc[state_name]['dcases'].plot(label='dcases')
    plt.legend()

### All States in this group surged early and peaked
- Pennsylvania stands out on the mobility measures but all have seen an upward trend, which makes sense given how far the mobility measures fell. A month from now will be interesting to see if trends return to or above the baseline (remember it was set in Jan and Feb)
- Mobility at the lows around Mid April

In [ ]:
ax = sns.lineplot(x="date", y="retail_and_recreation", data=df1)


In [ ]:
# The "Falling"
plot_mobility_cases('Illinois')
plot_mobility_cases('New York')
plot_mobility_cases('New Jersey')
plot_mobility_cases('Pennsylvania')
plot_mobility_cases('Massachusetts')

## States that have seen a recent increase in the rate of new cases
- Texas, Arizona and North Carolina have seen notable rises across the mobility measures
- California and Florida mobility is notably lower than the others in this group, both had early spikes in cases
- Texas, Arizona and North Carolina were relatively spared at the start of the pandemic, perhaps leading to false confidence
- The troughs in the mobility measures were notably shorter for some of the states

In [ ]:
plot_mobility_cases('Texas')
plot_mobility_cases('Arizona')
plot_mobility_cases('North Carolina')
plot_mobility_cases('Florida')
plot_mobility_cases('California')

## Covid Data EDA

### Creating 7 day moving average

### What states are seeing the biggest change in the number of daily cases? A measure often cited in the media is the 7 day moving average. It smooths out some of the daily fluctuations and allows us to look at longer chunks of time 

In [ ]:
# Setting up Moving average and moving average per capita for 7 days
df1['MovingAvg7'] = df1.loc[:,'dcases'].rolling(window=7).mean()
df1['MovingAvg7_percap'] = df1.loc[:,'cases_per_capita'].rolling(window=7).mean()

### Creating 2 date "snapshots"
- for later use in analyzing month to month differences

In [ ]:
df_latest = df1[df1['date']=='2020-08-07']
df_last_month = df1[df1['date']=='2020-07-12']

### Vizualizing 7 Day moving average of new cases for each state
- population size is obviously a big driver, but I see some states that aren't the highest pop as well
- let's dig deeper

In [ ]:
plt.figure()
x = df_latest.sort_values('MovingAvg7', axis=0, ascending=False)['MovingAvg7']
y = df_latest.sort_values('MovingAvg7', axis=0, ascending=False)['state']
plt.figure(figsize=(8, 15))
sns.barplot(x=x, y=y, orient='h')
#plt.xlim(0,0.3)  fix population on x access?
plt.xticks(rotation='vertical')
plt.title('7 Day Moving Average')
plt.show()

### Moving average per capita
- Making sure we're not seeing any population effects dominate our data

In [ ]:
plt.figure()
x = df_latest.sort_values('MovingAvg7_percap', axis=0, ascending=False)['MovingAvg7_percap']
y = df_latest.sort_values('MovingAvg7_percap', axis=0, ascending=False)['state']

plt.figure(figsize=(8, 15))
sns.barplot(x=x, y=y, orient='h')
#plt.xlim(0,0.3)  fix population on x access?
plt.xticks(rotation='vertical')
plt.title('7 Day Moving Average Per Capita')
plt.show()

### Plotting 7 day MA for Illinois, Texas, NY and California
- Illinois, New York, Texas and California are all high on the 7 day MA list, most likely because they were high on overall number of cases due to population
- Let's look at 7 day moving average over time for a few states
- Illinois and NYC both turned lower
- We're not capturing states that have a diminishing number of cases with our above analysis

In [ ]:
plt.figure()
plt.title('IL 7 Day MA')
df1[df1['state']=='Illinois']['MovingAvg7'].plot()
plt.legend()

plt.figure()
plt.title('TX 7 Day MA')
df1[df1['state']=='Texas']['MovingAvg7'].plot()
plt.legend()

plt.figure()
plt.title('NY 7 Day MA')
df1[df1['state']=='New York']['MovingAvg7'].plot()
plt.legend()

plt.figure()
plt.title('CA 7 Day MA')
df1[df1['state']=='California']['MovingAvg7'].plot()
plt.legend()

### Cases per day: 7 day MA vs Last Month
- let's look at a "snapshot" one month ago versus today to assess which states are seeing a concerning rise and a reassuring fall in new daily cases

In [ ]:
df_latest = df1[df1['date']=='2020-08-07']
df_last_month = df1[df1['date']=='2020-06-27']

df_latest['Month_MA_pc'] = df_latest.loc[:,'MovingAvg7_percap'].values - df_last_month.loc[:,'MovingAvg7_percap'].values
df_latest['Month_MA'] = df_latest.loc[:,'MovingAvg7'].values - df_last_month.loc[:,'MovingAvg7'].values

## Comparing Month to Month moving average
- We use the dates 5/12 and 6/12 and we take a snapshot of the 7 day moving average
- What can we tell about the states that are increasing?
- Can we find similar policies in common across groups of states?
- What about the states that have seen improvement, are there similarities?

In [ ]:
plt.figure()
x = df_latest.sort_values('Month_MA', axis=0, ascending=False)['Month_MA']
y = df_latest.sort_values('Month_MA', axis=0, ascending=False)['state']

plt.figure(figsize=(8, 15))
sns.barplot(x=x, y=y, orient='h')
#plt.xlim(0,0.3)  fix population on x access?
plt.xticks(rotation='vertical')
plt.title('Month over month change in 7 Day MA on 6/22 vs 5/22')
plt.show()

### Per cap MA's - controlling for population
- Some smaller states have a worrying rise
- Large population states were hit hard early

In [ ]:
plt.figure()
x = df_latest.sort_values('Month_MA_pc', axis=0, ascending=False)['Month_MA_pc']
y = df_latest.sort_values('Month_MA_pc', axis=0, ascending=False)['state']

plt.figure(figsize=(8, 15))
sns.barplot(x=x, y=y, orient='h')
#plt.xlim(0,0.3)  fix population on x access?
plt.xticks(rotation='vertical')
plt.title('Month over month change in 7 Day MA per capita on 6/12 vs 5/12')
plt.show()

# Cleaning

In [ ]:
df1.interpolate()

In [ ]:
df.interpolate()

In [ ]:
df1

In [ ]:
df_hmm = df1[['state','date','MovingAvg7','retail_and_recreation']]

In [ ]:
df_hmm_IL = df_hmm[df_hmm['state'] == 'Illinois']

df_hmm_IL['MovingAvg7'].describe()

In [ ]:
df_hmm_IL['diff_retail_and_recreation'] = df_hmm_IL['retail_and_recreation'].diff()
df_hmm_IL['diff_retail_and_recreation'] = df_hmm_IL['diff_retail_and_recreation'].apply(lambda x: 1 if x >= 0 else 0)

In [ ]:
df_hmm_IL['MA7'] = df_hmm_IL['MovingAvg7'].apply(lambda x: 1 if x >= 1000 else 0)

In [ ]:
df_hmm_IL['diff_retail_and_recreation'] = df_hmm_IL['diff_retail_and_recreation'].fillna(0)
df_hmm_IL['MA7'] = df_hmm_IL['MA7'].fillna(0)

In [ ]:
df_hmm_IL

In [ ]:
df_hmm_IL.to_csv('df_hmm_IL.csv')

In [ ]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
 
    return alpha
 
 
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta
 
 
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b}

In [ ]:
data = df
 
V = df['dcases'].values
 
# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)
 
# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))
 
# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))
 
print(baum_welch(V, a, b, initial_distribution, n_iter=100))
